In [23]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [24]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.now())
print(f"{current_date}")

2025-02-22 16:22:38.895718


In [25]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [26]:
from datetime import datetime, timedelta, timezone
current_date =current_date.floor("h")

In [27]:
current_date.to_datetime64()

numpy.datetime64('2025-02-22T16:00:00.000000000')

In [34]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=52)

In [35]:
fetch_data_to

Timestamp('2025-02-22 16:00:00')

In [36]:
fetch_data_from

Timestamp('2025-01-01 16:00:00')

In [37]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [38]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-01.
Loading data for 2024-01...
Total records: 2,964,624
Valid records: 2,911,483
Records dropped: 53,141 (1.79%)
Successfully processed data for 2024-01.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!


In [39]:
rides

,pickup_datetime,pickup_location_id
37338,2025-01-02 05:29:47,2
2599114,2025-01-04 23:23:27,2
458517,2025-01-06 22:30:16,2
3998107,2025-02-12 16:25:44,2
4304808,2025-02-15 16:56:40,2
...,...,...
4942067,2025-02-22 15:58:19,263
4941771,2025-02-22 15:58:33,263
4945552,2025-02-22 15:58:35,263
4939684,2025-02-22 15:59:07,263


In [40]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [41]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-01 16:00:00,2,0
1,2025-01-01 17:00:00,2,0
2,2025-01-01 18:00:00,2,0
3,2025-01-01 19:00:00,2,0
4,2025-01-01 20:00:00,2,0
...,...,...,...
320731,2025-02-22 11:00:00,263,130
320732,2025-02-22 12:00:00,263,152
320733,2025-02-22 13:00:00,263,143
320734,2025-02-22 14:00:00,263,111


In [42]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320736 entries, 0 to 320735
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         320736 non-null  datetime64[ns]
 1   pickup_location_id  320736 non-null  int16         
 2   rides               320736 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 3.7 MB


In [43]:
ts_data.tail()

,pickup_hour,pickup_location_id,rides
320731,2025-02-22 11:00:00,263,130
320732,2025-02-22 12:00:00,263,152
320733,2025-02-22 13:00:00,263,143
320734,2025-02-22 14:00:00,263,111
320735,2025-02-22 15:00:00,263,141


In [46]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')

# Initialize connection to Hopsworks
project = hopsworks.login(
    api_key_value=api_key,
    project=project_name
)
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-02-22 16:24:48,433 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-02-22 16:24:48,459 INFO: Initializing external client
2025-02-22 16:24:48,459 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 16:24:49,486 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Successfully connected to Hopsworks project: nolanphi


In [47]:
feature_store = project.get_feature_store()

In [48]:
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

In [50]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 320736/320736 | Elapsed Time: 00:36 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)